In [161]:
# pip install biopython

In [162]:
import datetime
from Bio.Seq import Seq, back_transcribe

# 1. Class that describes hamsters.

Python class for hamsters.\
For creation "hamster" object it is nedeed to know its name and age (in months).\
Optionally, one can specify the type of hamster (for example, Syrian or Chinese). Default "Not defined".

In [137]:

class Hamster:
    '''
    Python class for hamsters.
    '''

    def __init__(self, name, age, hamster_type='Not defined'):
        '''
        For creation "hamster" object it is nedeed to know its name and age (in months).
        Optionally, one can specify the type of hamster (for example, Syrian or Chinese). Default "Not defined".
        '''
        self.name = name
        self.hamster_type = hamster_type
        self.age = age
        
        if self.age <= 4:
            self.stage = 'young'            
            
        elif self.age >= 12:
            self.stage = 'old'
        
        else:
            self.stage = 'adult'
        

    def life_stage(self):
        '''
        Method returns the age of a hamster: young, adult or old.
        '''        
        return f"{self.name} is an {self.stage} hamster. It's {self.age} months."


    def is_active(self):
        '''
        Checks if the hamster is probably asleep now. The system time is used. Return the string with state.
        '''
        time_now = datetime.datetime.now().hour
        
        if time_now > 7 and time_now < 20:
            return f'{self.name} is probably sleeping now.'
            
        else:
            return f'{self.name} is probably active now and wants treats.'


    def properties(self):
        '''
        Return string with the basic parameters of the hamster.
        '''
        if self.hamster_type == 'Not defined':            
            return f"The hamster\'s name is {self.name}. It\'s {self.stage} hamster, {self.age} months."
        else:
            return f"The hamster\'s name is {self.name}. It\'s {self.stage} {self.hamster_type} hamster, {self.age} months."

In [138]:
Bun = Hamster('Bun', 18, 'Djungarian hamster')
print(Bun.is_active())
print(Bun.properties())
print(Bun.life_stage())

Bun is probably active now and wants treats.
The hamster's name is Bun. It's old Djungarian hamster hamster, 18 months.
Bun is an old hamster. It's 18 months.


# 2. Class that describes RNA sequence data

Class (rooted from str class) with two methods:
- `RNA_translation` - Translation - returns a string corresponding to the protein from the RNA, according to the standard code, using Biopython.
- `RNA_to_DNA` - Reverse transcription method - returns a string corresponding to the DNA from RNA, using Biopython

In [163]:
class RNASequence(str):
    '''
    Class for operations with RNA sequences. Parent class - str.
    '''
    acid_type = 'RNA'
    
    def __init__(self, rna_sequence):
        self.rna_sequence = rna_sequence
        
    def RNA_translation(self):
        '''
        Translation - returns a string corresponding to the protein from the RNA,
        according to the standard code, using Biopython.
        '''
        self.protein = Bio.Seq.translate(self.rna_sequence)
        return self.protein
        
    def RNA_to_DNA(self):
        '''
        Reverse transcription method - returns a string corresponding to the DNA from RNA, using Biopython.
        '''
        self.template_dna = Bio.Seq.back_transcribe(self.rna_sequence)
        return self.template_dna

In [164]:
# test for RNASequence class

rna_seq = RNASequence('AUGGCCAUUGUAAUGGGCCGCUGAAAGGGUGCCCGAUAG')
print(rna_seq.RNA_translation())
print(rna_seq.RNA_to_DNA())

rna_seq_2 = RNASequence('AUGC')
print(rna_seq_2.RNA_translation())
print(rna_seq_2.RNA_to_DNA())

MAIVMGR*KGAR*
ATGGCCATTGTAATGGGCCGCTGAAAGGGTGCCCGATAG
M
ATGC


# 3. 

In [659]:
class PositiveSet(set):
    '''
    Sets with only positive numbers.
    Two methods supports: add and update (but not |=).
    '''
    
    def __init__(self, old_numbers):
        '''
        Initializes PositiveSet, adding only positive numbers using the add method.
        '''
        for number in old_numbers:
            self.add(number)
    
    def add(self, element):
        '''
        Check if number for adding is positive. 
        If so, it adds it to the PositiveSet using the regular set 'add' method.
        '''
        if element >= 0:
            super().add(element)
        
    def update(self, *args):
        '''
        Update PositiveSet using positive values (PositiveSet 'add' method) from the given args.
        '''
        if len(args) == 1:
            for element in args[0]:
                self.add(element)
        else:
            for added_set in args[:-1]:
                for element in added_set:
                    self.add(element)
    

In [660]:
# test for PositiveSet.add() method
ordinary_set = set([1, -2, 3, 4, -5, -6, 7])
print(ordinary_set)

new_set = PositiveSet(ordinary_set)
print(new_set)

new_set.add(-5)
print(new_set)

{1, 3, 4, 7, -6, -5, -2}
PositiveSet({1, 3, 4, 7})
PositiveSet({1, 3, 4, 7})


In [661]:
# test for PositiveSet.update() method

new_set.update([-4, 87])
print(new_set)

y = {14, -15}
x = {10, -12}

new_set.update(x, y)

print(new_set)

PositiveSet({1, 3, 4, 7, 87})
PositiveSet({1, 3, 4, 7, 10, 87})
